# Turing Machines

[On Computable Numbers, with an Application to the Entscheidungsproblem](https://people.math.ethz.ch/~halorenz/4students/Literatur/TuringFullText.pdf)  
Proceedings of the London Mathematical Society Series 2, 42, 230-265  
Turing, A. M. (1937)  

## Long Multiplication

Multiply 123 by 45.  
You need your times tables for 1 to 9.  
They are a list of instructions.

```
 123
x 45
----
 615
4920
----
5535
```

```
123x45=
```

```
123x45=615,4920=5535
```

## End